# 食べログ つくば

## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pickle
from tqdm import tqdm
import pandas as pd

## listPageのURLリスト作成

In [2]:
pages = range(1, 61)

In [3]:
listpages = [f"https://tabelog.com/ibaraki/A0802/A080201/rstLst/{page}/" for page in pages]

## 詳細ページのURLを取得し、リストを作成

In [4]:
store_url_list = []
for page in tqdm(listpages):
    response = requests.get(page)
    sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')
    store_urls = []
    for link in soup.select('a.list-rst__rst-name-target'):
        store_urls.append(link['href'])
    store_url_list.extend(store_urls)

  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [02:09<00:00,  2.15s/it]


In [5]:
# 一旦保存
with open('store_url_list_tsukuba.pkl', 'wb') as f:
    pickle.dump(store_url_list, f)

## スクレイピング

In [6]:
data = []
for i, store_url in tqdm(enumerate(store_url_list), total=len(store_url_list)):
    try:
        response = requests.get(store_url)
        sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')
        details = {
            '店名': ' - ',
            '住所': ' -',
            '電話番号': ' - ',
            '予約・お問い合わせ': ' - ',
        }
        table = soup.find(id='contents-rstdata')
        th_elements = table.find_all('th')
        td_elements = table.find_all('td')

        # テキスト取得
        th_texts = [th.text.replace('\n', '').replace(' ', '') for th in th_elements]
        td_texts = [td.text for td in td_elements]

        # 辞書マッピング
        data_map = dict(zip(th_texts, td_texts))

        # データ抽出
        details['店名'] = data_map.get('店名', ' - ').replace('\n', '').strip()
        details['住所'] = data_map.get('住所', ' - ').replace('\n', '').strip().split('大きな地図')[0]
        details['電話番号'] = data_map.get('電話番号', ' - ').replace('\n', '').strip()
        details['予約・お問い合わせ'] = data_map.get('予約・お問い合わせ', ' - ').replace('\n', '').strip()
        data.append(details)
    except Exception as e:
        print(f"Error processing {i}: {stre_url}: {e}")
        pass

100%|██████████| 1200/1200 [34:16<00:00,  1.71s/it]


## CSVファイルに書き出し

In [7]:
df = pd.DataFrame(data)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.to_csv('tsukuba.csv', index=False, encoding='utf-8-sig')